<a href="https://colab.research.google.com/github/MDankloff/ScriptAIRbnb.R/blob/main/COMPAS_data_inspection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")
import os

# Specify the directory path
directory_path = '//Users/mirthedankloff/Documents/XAI_Bias_Error/COMPAS'

# Traverse the directory and print the names of all files
for dirname, _, filenames in os.walk(directory_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))


LOAD COMPAS data

In [91]:
compas_s = pd.read_csv("/content/final_preprocessed_compas(2).csv")
ofercompas = pd.read_csv("/content/propublica_data_for_fairml.csv")
originalcompas = pd.read_csv("/content/compas-scores-two-years-violent.csv")

In [92]:
target = "Two_yr_Recidivism"
X = ofercompas.drop([target],axis=1)
Y = ofercompas[target]

TRAIN  Model

In [93]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, shuffle = True, stratify = Y)

model = RandomForestClassifier()
model.fit(X_train, Y_train)

#predictions
Yhat = model.predict(X_test)
len(Yhat)


1852

COLLECT the errors

In [94]:
predictions = pd.DataFrame()
predictions['predicted_class'] = Yhat.tolist()
predictions = predictions.set_index(X_test.index)
predictions['true_class'] = Y_test

#accuracy
accuracy = accuracy_score(predictions['true_class'], predictions['predicted_class'])
print(f"Accuracy: {accuracy:.4f}")

#confusion matrix
cm = confusion_matrix(predictions['true_class'], predictions['predicted_class'])
print(cm)


Accuracy: 0.6604
[[736 273]
 [356 487]]


In [101]:
predictions['errors'] = abs(predictions['predicted_class'] - predictions['true_class'])
predictions['errors'] = predictions['errors'].replace(2.0, 1.0)


data_pred = X_test.merge(predictions, how= 'outer', left_index = True, right_index = True)

errors = data_pred['errors']
ofercompas_out = data_pred.drop(['predicted_class', 'true_class', 'errors'], axis=1)

display(data_pred.columns)
print(len(data_pred))
data_pred[data_pred['errors']==1]

Index(['Number_of_Priors', 'score_factor', 'Age_Above_FourtyFive',
       'Age_Below_TwentyFive', 'African_American', 'Asian', 'Hispanic',
       'Native_American', 'Other', 'Female', 'Misdemeanor', 'predicted_class',
       'true_class', 'errors'],
      dtype='object')

1852


,Number_of_Priors,score_factor,Age_Above_FourtyFive,Age_Below_TwentyFive,African_American,Asian,Hispanic,Native_American,Other,Female,Misdemeanor,predicted_class,true_class,errors
5093,1,0,0,0,1,0,0,0,0,1,1,0,1,1
2101,3,0,1,0,1,0,0,0,0,0,1,1,0,1
5831,15,1,1,0,0,0,0,0,0,1,0,1,0,1
6137,0,0,0,1,0,0,1,0,0,0,1,1,0,1
4414,2,0,0,0,0,0,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6100,1,1,0,0,1,0,0,0,0,1,0,0,1,1
1182,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2463,1,0,1,0,1,0,0,0,0,0,1,0,1,1
5628,0,1,0,1,1,0,0,0,0,0,1,0,1,1


Selecting 4 labels (FP, FN, TP, TN)

In [104]:
def get_error_type(row):
    true_class = row['true_class']
    predicted_class = row['predicted_class']

    # Calculate confusion matrix for each instance
    cm = confusion_matrix([true_class], [predicted_class])

    # Extract TP, FP, FN, TN
    TP = cm[0, 0] if len(set([true_class, predicted_class])) == 2 and true_class == 1 else 0
    FP = cm[0, 1] if len(set([true_class, predicted_class])) == 2 and true_class == 0 else 0
    FN = cm[1, 0] if len(set([true_class, predicted_class])) == 2 and true_class == 1 else 0
    TN = cm[1, 1] if len(set([true_class, predicted_class])) == 2 and true_class == 0 else 0

    # Determine the error type

    row['TP'] = TP
    row['FP'] = FP
    row['FN'] = FN
    row['TN'] = TN

    return row


# Apply the function to each row
data_pred = data_pred.apply(get_error_type, axis=1)

# Display the result
print(data_pred)
display(data_pred.columns)


      Number_of_Priors  score_factor  Age_Above_FourtyFive  \
121                  0             0                     0   
3627                 0             0                     1   
5093                 1             0                     0   
2281                 2             0                     0   
2101                 3             0                     1   
...                ...           ...                   ...   
1280                11             1                     1   
2463                 1             0                     1   
5628                 0             1                     0   
2828                24             1                     0   
3834                 0             0                     0   

      Age_Below_TwentyFive  African_American  Asian  Hispanic  \
121                      0                 0      0         0   
3627                     0                 0      0         0   
5093                     0                 1      0         

Index(['Number_of_Priors', 'score_factor', 'Age_Above_FourtyFive',
       'Age_Below_TwentyFive', 'African_American', 'Asian', 'Hispanic',
       'Native_American', 'Other', 'Female', 'Misdemeanor', 'predicted_class',
       'true_class', 'errors', 'TP', 'FP', 'FN', 'TN'],
      dtype='object')

Scatterplot 4 labels (FP, FN, TP, TN)

PCA 4 labels (FP, FN, TP, TN)